In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/creditcard.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [7]:
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


# xgboost

hyper parameter
---------
|하이퍼 파라미터|설명|
|----|--|
|eta(default=0.3)|학습률, weak learner의 반영 수준을 나타냄, 범위는 0-1로 클수록 모형 업데이트 속도 faster, overfitting 이슈 발생 가능성 high|
|num_boost_around(default=10)|학습에 활용될 weak learners의 반복수|
|min_child_weight(default=1)|leaf node에 포함되는 최소 관측치의 수, 작은 값을 가질수록 overfitting 가능성 높음, 범위 : 0~|
|gamma(default=0)|leaf node의 추가 분할을 결정할 최소손실 감소값, 해당값보다 손실이 크게 감소할 때 분리, 값이 클수록 과적합 감소 효과, 범위 : 0~|
|5. max_depth(default=6)|트리의 최대 깊이 설정, 0으로 지정하면 깊이의 제한 x, overfitting에 가장 민감하게 작용하는 파라미터 중 하나, 범위 : 0~|
|6. sub_sample(default=1)|학습 시 데이터 샘플링 비율을 지정, 일반적으로 0.5~1 사이의 값을 사용, 범위 : 0~1|
|7. colsample_bytree(default=1)|트리 생성에 필요한 feature의 샘플링에 사용, feature가 많을 때 과적합 조절에 사용, 범위 : 0~1|
|8. lambda(default=1)|L2 Regularization 적용값, feature 개수가 많을 때 적용 검토, 클수록 과적합 감소 효과|
|9. alpha(default=0)|L1 Regularization 적용값, feature 개수가 많을 때 적용 검토, 클수록 과적합 감소 효과|
|10. scale_pos_weight(default=1)|불균형 데이터셋의 균형을 유지|

In [ ]:
# 데이터 전처리
# 불필요한 열 제거
data.drop(['Time'], axis=1, inplace=True)
# 타겟 변수와 특징 변수 나누기
target = 'Class'
features = [col for col in data.columns if col != target]

# 훈련 데이터와 검증 데이터 나누기
train_data, val_data, train_target, val_target = train_test_split(
    data[features], data[target], test_size=0.2, random_state=42)

In [10]:
# XGBoost 모델 학습시키기
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}
dtrain = xgb.DMatrix(train_data, train_target)
dval = xgb.DMatrix(val_data, val_target)
model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dval, 'val')],
                  early_stopping_rounds=10, verbose_eval=10)

# 검증 데이터에서 예측하기
val_pred = model.predict(dval)

# 검증 데이터에서 AUC 평가하기
val_auc = roc_auc_score(val_target, val_pred)
print(f'검증 데이터에서 AUC: {val_auc:.4f}')

[0]	val-auc:0.92327
[10]	val-auc:0.92320
[12]	val-auc:0.92320
검증 데이터에서 AUC: 0.9232


# LGBM

hyper paramater
---
|하이퍼파라미터|설명|
|--------------|----|
|objective	|목적 함수. 이진 분류를 수행하므로 'binary'로 설정|
|metric|	모델 성능 평가 지표. AUC 사용|
|learning_rate|	학습률. 너무 높으면 수렴하지 못하고, 너무 낮으면 수렴 속도가 느리므로 적절한 값을 설정|
|num_leaves|	결정 트리의 최대 잎 노드 개수. 과적합 방지를 위해 적절한 값을 설정|
|max_depth|	결정 트리의 최대 깊이. None으로 설정하면 깊이에 제한이 없어 과적합 가능성이 높아짐|
|min_child_samples|	잎 노드에 최소한으로 필요한 데이터 수. 적은 값은 깊은 트리를 만들어 과적합 가능성이 있음|
|subsample	|각 트리 학습에 사용할 데이터 샘플링 비율|
|colsample_bytree	|각 트리 학습에 사용할 특성 샘플링 비율|
|reg_alpha	|L1 정규화 파라미터|
|reg_lambda	|L2 정규화 파라미터|
|n_jobs	|학습에 사용할 CPU 코어 수. -1로 설정하면 모든 코어 사용|
|random_state	|랜덤 시드 값|

In [19]:
# LightGBM 모델 학습시키기
params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'n_jobs': -1,
    'random_state': 42
}
dtrain = lgb.Dataset(train_data, train_target)
dval = lgb.Dataset(val_data, val_target)
model = lgb.train(params, dtrain, num_boost_round=1000, valid_sets=[dtrain, dval],
                  early_stopping_rounds=10, verbose_eval=10)

# 검증 데이터에서 예측하기
val_pred = model.predict(val_data)

# 검증 데이터에서 AUC 평가하기
val_auc = roc_auc_score(val_target, val_pred)
print(f'검증 데이터에서 AUC: {val_auc:.4f}')

[LightGBM] [Info] Number of positive: 394, number of negative: 227451
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.117018 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 227845, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001729 -> initscore=-6.358339
[LightGBM] [Info] Start training from score -6.358339
Training until validation scores don't improve for 10 rounds
[10]	training's auc: 0.988588	valid_1's auc: 0.96869
Early stopping, best iteration is:
[7]	training's auc: 0.987413	valid_1's auc: 0.970097
검증 데이터에서 AUC: 0.9701


In [15]:
# LightGBM 모델 학습시키기
params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'n_jobs': -1,
    'random_state': 42
}
dtrain = lgb.Dataset(train_data, train_target)
dval = lgb.Dataset(val_data, val_target)
model = lgb.train(params, dtrain, num_boost_round=1000, valid_sets=[dtrain, dval],
                  early_stopping_rounds=10, verbose_eval=10)

# 검증 데이터에서 예측하기
val_pred = model.predict(val_data)

# 검증 데이터에서 AUC 평가하기
val_auc = roc_auc_score(val_target, val_pred)
print(f'검증 데이터에서 AUC: {val_auc:.4f}')

[LightGBM] [Info] Number of positive: 394, number of negative: 227451
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.053167 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 227845, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001729 -> initscore=-6.358339
[LightGBM] [Info] Start training from score -6.358339
Training until validation scores don't improve for 10 rounds
[10]	training's auc: 0.988588	valid_1's auc: 0.96869
Early stopping, best iteration is:
[7]	training's auc: 0.987413	valid_1's auc: 0.970097
검증 데이터에서 AUC: 0.9701


#RF

hyper parameter
---
|하이퍼파라미터|설명|
|--------------|----|
|n_estimators|생성할 결정 트리 개수|
|max_depth|	결정 트리의 최대 깊이|
|random_state|	랜덤 시드 값|


In [ ]:
# 데이터 전처리
# 불필요한 열 제거
data.drop(['Time'], axis=1, inplace=True)
# 타겟 변수와 특징 변수 나누기
target = 'Class'
features = [col for col in data.columns if col != target]

# 훈련 데이터와 검증 데이터 나누기
train_data, val_data, train_target, val_target = train_test_split(
    data[features], data[target], test_size=0.2, random_state=42)

In [23]:
# 랜덤 포레스트 모델 생성 및 학습
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
model.fit(train_data, train_target)

# 예측 결과 출력
val_pred = model.predict(val_data)
print(classification_report(val_target, val_pred))
print(confusion_matrix(val_target, val_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.97      0.79      0.87        98

    accuracy                           1.00     56962
   macro avg       0.99      0.89      0.93     56962
weighted avg       1.00      1.00      1.00     56962

[[56862     2]
 [   21    77]]


In [24]:
# 랜덤 포레스트 모델 생성 및 학습
model = RandomForestClassifier(n_estimators=200, max_depth=6, random_state=42)
model.fit(train_data, train_target)

# 예측 결과 출력
val_pred = model.predict(val_data)
print(classification_report(val_target, val_pred))
print(confusion_matrix(val_target, val_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.97      0.73      0.84        98

    accuracy                           1.00     56962
   macro avg       0.99      0.87      0.92     56962
weighted avg       1.00      1.00      1.00     56962

[[56862     2]
 [   26    72]]


In [25]:
# 랜덤 포레스트 모델 생성 및 학습
model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=0)
model.fit(train_data, train_target)

# 예측 결과 출력
val_pred = model.predict(val_data)
print(classification_report(val_target, val_pred))
print(confusion_matrix(val_target, val_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.97      0.77      0.86        98

    accuracy                           1.00     56962
   macro avg       0.99      0.88      0.93     56962
weighted avg       1.00      1.00      1.00     56962

[[56862     2]
 [   23    75]]
